In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import pickle

In [2]:
# Read in our input data
train = pd.read_csv('../../Dataset/train/train.csv')
test = pd.read_csv('../../Dataset/test/test.csv')


# This prints out (rows, columns) in each dataframe
print('Train shape:', train.shape)
print('Test shape:', test.shape)


id_train = train['id'].values
y = train.target.values
id_test = test['id'].values

Train shape: (595212, 59)
Test shape: (892816, 58)


In [3]:
train['countNAs'] = train.isin(['-1']).sum(axis=1)
test['countNAs'] = test.isin(['-1']).sum(axis=1)

train = train.drop(['id','target'], axis=1)
test = test.drop(['id'], axis=1)

In [8]:
feature_columns = list(train.columns)
cols2id = {col:str(i+1) for i,col in enumerate(feature_columns)}

In [4]:
train['score'] = y
train['srch_id'] = 1
test['score'] = 1
test['srch_id'] = 1

In [15]:
kfold=22
cval = KFold(n_splits=kfold, random_state=42)

In [16]:
val = val.reset_index()
for split in enumerate(cval.split(val)):
    val.ix[list(split[1][1]), 'srch_id'] = split[0] + 1

/home/arsh/anaconda3/envs/deepLearn/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


KeyError: '[178564 178565 178566 ..., 202912 202913 202914] not in index'

In [16]:
test = test.reset_index()
for split in enumerate(cval.split(test)):
    test.ix[list(split[1][1]), 'srch_id'] = split[0] + 1

/home/arsh/anaconda3/envs/deepLearn/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


In [7]:
train = train.reset_index()
for split in enumerate(cval.split(train)):
    train.ix[list(split[1][1]), 'srch_id'] = split[0] + 1

/home/arsh/anaconda3/envs/deepLearn/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


In [6]:
train, val = train_test_split(train, test_size=0.7, random_state=42)

In [5]:
train, val = train_test_split(train, test_size=0.3, random_state=42)

In [8]:
def f(x):
    if x.name in feature_columns:
        return cols2id[x.name] + ':' + x.astype(str)
    elif x.name == 'srch_id':
        return 'qid:' + x.astype(str)
    else:
        return x

(train.apply(lambda x: f(x))[['score','srch_id'] + feature_columns]
  .to_csv('train_1list_30per.csv', sep=' ', index=False, header=None)
)

In [ ]:
def f(x):
    if x.name in feature_columns:
        return cols2id[x.name] + ':' + x.astype(str)
    elif x.name == 'srch_id':
        return 'qid:' + x.astype(str)
    else:
        return x

(val.apply(lambda x: f(x))[['score','srch_id'] + feature_columns]
  .to_csv('val_1list_30per.csv', sep=' ', index=False, header=None)
)

In [17]:
def f(x):
    if x.name in feature_columns:
        return cols2id[x.name] + ':' + x.astype(str)
    elif x.name == 'srch_id':
        return 'qid:' + x.astype(str)
    else:
        return x

(test.apply(lambda x: f(x))[['score','srch_id'] + feature_columns]
  .to_csv('test_38lists.csv', sep=' ', index=False, header=None)
)

In [21]:
preds = pd.read_csv('./gcloud_submit/predictions_listNet_50lists_val.csv', index_col=0)

In [26]:
gini_normalized(val['score'].values, preds['probability'].values)

0.00077335934521688778

In [27]:
preds

,probability
id,
0,0.650519
1,0.634076
2,0.480112
3,0.570828
4,0.598300
5,0.531592
6,0.646798
7,0.588022
8,0.559865


In [6]:
# Define the gini metric - from https://www.kaggle.com/c/ClaimPredictionChallenge/discussion/703#5897
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

# Create an XGBoost-compatible metric from Gini

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return 'gini', gini_score

def gini_lgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return 'gini', 1-gini_score, False